# Amazon Braket Hybrid Jobs 入門

このチュートリアルでは、Amazon Braket Hybrid Job を実行する方法を紹介します。始めに、1 つの量子ビットと 1 つのゲートのみを持つシンプルな回路を考えます。

## 内容
* セットアップ
* ジョブで実行するアルゴリズムスクリプトの作成
* スクリプトや関数の実行方法
* Braket のシミュレータや QPU を使ったジョブの作成
* ジョブの状態の確認
* ジョブの結果を保存
* 特定の AWS セッションの使用
* QPU 上の優先的なジョブの実行
* ローカルジョブを使用したスクリプトの迅速なテストおよびデバッグ
* Braket コンソールを使用したジョブの作成

## Braket ジョブを実行するためのセットアップ

Amazon Braket Hybrid Jobsを初めて使用する場合、[適切な権限](https://docs.aws.amazon.com/braket/latest/developerguide/braket-manage-access.html#about-amazonbraketjobsexecution)を持つ IAM ロールを作成する必要があります。このロールはジョブがアルゴリズムを実行する際、お客様の代わりになってアクションを実行します。例えば、ジョブの結果を返すために S3 にアクセスさせるかどうかなどをこの IAM ロールで指定します。ロールの作成 / 確認を行うには、Braket Console の左メニューから Permissions タブにアクセスしてください。

## アルゴリズムスクリプトの作成

Braket Job を作成するには、まず実行するための Python スクリプトが必要です。今回の例では、`algorithm_script.py` にあたります。以下のセルにその中身を載せています。

これを見ると、各回路には、ランダムな角度の $X$ 回転ゲートが 1 つだけあり、角度を変えて 5 回繰り返されることがわかります。なお、このアルゴリズムスクリプトでは、バックエンドの Amazon Braket デバイス の ARN を明示的に指定していません。代わりに、ジョブの作成時にアルゴリズムスクリプトに渡される環境変数 `os.environ["AMZN_BRAKET_DEVICE_ARN"]` として指定しています。

#### このセルはアルゴリズムスクリプトのコピーです。

```python

import os
import numpy as np

from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.jobs import save_job_result
from braket.tracking import Tracker

t = Tracker().start()

print("Test job started!")

# Use the device declared in the creation script
device = AwsDevice(os.environ["AMZN_BRAKET_DEVICE_ARN"])

counts_list = []
angle_list = []
for _ in range(5):
    angle = np.pi * np.random.randn()
    random_circuit = Circuit().rx(0, angle)

    task = device.run(random_circuit, shots=100)
    counts = task.result().measurement_counts

    angle_list.append(angle)
    counts_list.append(counts)
    print(counts)

# Save the variables of interest so that we can access later
save_job_result({"counts": counts_list, "angle": angle_list, "estimated cost": t.qpu_tasks_cost() + t.simulator_tasks_cost()})

print("Test job completed!")
```

## ジョブを作成する

スクリプトが書けたら、`AwsQuantumJob`で Braket Job を作成することができます。ジョブが作成されると、Amazon Braket はジョブインスタンス (EC2 ベース) を起動し、Docker コンテナ上でアルゴリズムスクリプトを実行します。その他の設定は引数として指定することができます。ジョブをカスタマイズする方法については、[開発者ガイド](https://docs.aws.amazon.com/braket/latest/developerguide/what-is-braket.html)や他のサンプルノートブックを参照してください。

今回の例では、`AwsQuantumJob` に以下の設定を行っています。
- <b>device</b>: ジョブで使われる Braket シミュレータもしくは QPU の ARN です。アルゴリズムスクリプトへの環境変数として渡されます。
- <b>source_module</b>:　アルゴリズムスクリプトを含むファイルもしくは Python モジュールへのパスです。Braket Job を実行するコンテナにアップロードされます。
- <b>wait_until_complete (オプション)</b>: True の場合、ジョブが完了するまで待機し、ローカルのコンソールにログを出力します。False の場合、ジョブを非同期に実行します。デフォルト値は False です。

In [1]:
from braket.aws import AwsQuantumJob

In [2]:
# このセルの実行には 5 分ほどかかります
job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
    wait_until_complete=True,
)

Initializing Braket Job: arn:aws:braket:us-east-1:700863243650:job/braket-job-default-1684033674250
.........................
2023-05-14 03:10:08,379 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-05-14 03:10:08,393 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-05-14 03:10:08,405 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-05-14 03:10:08,416 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {},
    "current_host": "algo-1",
    "framework_module": null,
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {},
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {},
    "input_dir": "/opt/ml/input",
    "is_master": true,
    "job_name": "bef49f5c-f5d7-43dc-9849-d08a5fc489a1",
    "log_level": 20,
    "master_hostname": "algo-1",
    "model_di

今回の例では、アルゴリズムは 1 つのファイルで定義されているため、`source_module` は `algorithm_script.py` です。アプリケーションによっては、ソースモジュールを他の方法で設定できます。例えば、ジョブの開始時に `algorithm_script.py` の一部だけを実行したい場合、その部分を `starting_function()` としてパッケージ化することも可能です。そして、`entry_point` という引数を追加して、その関数をエントリポイントとして割り当てます。

In [3]:
source_module = "algorithm_script.py"
entry_point = "algorithm_script:starting_function"

アルゴリズムスクリプトが他のファイルと依存関係を持つ場合、それらをすべて 1 つのフォルダ、例えば `algorithm_folder` に置くことができます。この場合、引数は次のようになります。

In [4]:
source_module = "algorithm_folder"
entry_point = "algorithm_folder.algorithm_script:starting_function"

## ジョブの状態の確認と結果の取得

Braket Job の状態は `job.state()` を呼び出すことで確認できます。状態は "QUEUED", "RUNNING", "FAILED", "COMPLETED", "CANCELLING",  "CANCELLED" のいずれかとなります。

In [5]:
job.state()

'COMPLETED'

完了 ("COMPLETED") したら、`job.result()` を用いて結果を取得することができます。ログとメタデータも `job.logs()` と `job.metadata()` で取得できます。ジョブオブジェクトの参照を失った場合は、ジョブ ARN を使用して `job=AwsQuantumJob("your-job-arn")` としていつでも再定義できます。ジョブ ARN は Amazon Braket Console で確認できます。デフォルトでは、ジョブ ARN は "`arn:aws:braket:<region>:<account_id>:job/<job_name>`" の形式です。

In [6]:
results = job.result()  # job.state() = "COMPLETED" となれば結果を返します
print("counts: ", results["counts"])
print("angles: ", results["angles"])

counts:  [{'1': 82, '0': 18}, {'1': 96, '0': 4}, {'0': 99, '1': 1}, {'1': 34, '0': 66}, {'0': 100}]
angles:  [2.2579830553408797, -2.8787011487593785, 0.09387639599223568, 5.137803481300752, -0.12194465950382408]


In [8]:
# print(job.logs())  # ログを出力したい場合はコメントアウトを外してください

結果をローカルの環境にダウンロードすることもできます。

In [9]:
job.download_result()  # 結果をローカル環境にダウンロード

In [10]:
print("Task Summary")
print(job.result()['task summary'])
print('Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).')
print(f"Estimated cost to run tasks in this job: {job.result()['estimated cost']} USD")

Task Summary
{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 500, 'tasks': {'COMPLETED': 5}, 'execution_duration': 0.041, 'billed_execution_duration': 15.0}}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run tasks in this job: 0.01875 USD


## QPU 上で優先的にジョブを実行する

Braket Jobs を使用すると、Amazon Braket で利用できる全ての QPU でハイブリッドアルゴリズムを実行することができます。QPU をデバイスとして選択すると、ジョブの実行中、QPU に優先的にアクセスできるようになります。ジョブの一部として作成される量子タスクは、デバイスのキューにある他のタスクよりも先に実行されます。これにより、特定のタスクが遅延したり、ジョブ実行中にデバイスのキャリブレーションに変動が生じるリスクを低減します。

`AwsQuantumJob.create()` の device 引数を変更することで、SV1 シミュレータを QPU にシームレスに入れ替えることができます。例えば、以下のコードでは、Rigetti Aspen-M-3 デバイス上で優先アクセス権を持つジョブを作成します。

<div class="alert alert-block alert-info">
注意: 以下のセルは Rigetti Aspen-M-3 デバイスを使用しています。コメントを解除して実行する前に、デバイスが現在利用可能であることを確認してください。QPU の利用可能時間帯 は Amazon Braket コンソール画面の <a href="https://us-west-1.console.aws.amazon.com/braket/home?region=us-west-1#/devices">Devices ページ</a>で確認できます。
</div>

In [ ]:
# qpu_job = AwsQuantumJob.create(
#     device="arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-3",
#     source_module="algorithm_script.py",
#     wait_until_complete=False,
# )

ジョブを作成すると、Amazon Braket はジョブを初期化する前に、QPU が実行可能になるのを待ちます。なお、Braket Job は、デバイスが利用可能な AWS リージョンを自動的に選択します。前述のように、指定されたデバイスは環境変数 `AMZN_BRAKET_DEVICE_ARN` でジョブに提供されます。スクリプト `algorithm_script.py` はこの変数を使って、使用する Braket デバイスを選択します。

変分アルゴリズムでは通常、固定されたパラメータ付き回路が持つパラメータを更新する最適化処理が行われます。このようなアルゴリズムを QPU で実行する場合、パラメトリックコンパイルによってジョブの性能を向上させることができます。その際必要なのは、スクリプトで free parameters を使用してパラメータ付き回路を定義するだけです。このようにすると、Braket は回路を一度コンパイルし、コンパイル済み回路を管理するようになります。パラメータを更新する際は再コンパイルする必要がないため、実行時間が短縮されます。パラメータ化された回路を使用するスクリプトの例は `algorithm_script_parametrized_circuit.py` にあります。このスクリプトを`algorithm_script.py` の代わりに使用し、QPU にジョブを投げることでパラメトリックコンパイルを使用できます。Hybrid Jobs での [free parameters](https://docs.aws.amazon.com/braket/latest/developerguide/braket-constructing-circuit.html#braket-gates) と[パラメトリックコンパイル](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs.html)の使い方については、ドキュメントをご参照ください。

## AWS セッションについて

Braket Jobs が Amazon S3 に結果を保存するデフォルトの場所は、AWS のセッション情報を提供することでカスタマイズすることができます。S3 バケットの名前は "amazon-braket-" で始まる必要があり、作成されるジョブと同じリージョンにある必要があります。

In [11]:
from braket.aws import AwsSession

# Set Amazon S3 bucket
aws_session = AwsSession(default_bucket="amazon-braket-bucket-name")

この S3 バケットを使って Braket Job を作成するには、`AwsQuantumJob.create()` の引数に `aws_session` を渡します。
```
job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
    aws_session=aws_session # using specific S3 bucket
)
```

## ローカルジョブを利用してデバッグする

コードのテストやデバッグをより迅速に行うために、お客様の環境でジョブをローカルに実行することができます。この機能を使用するには、ローカル環境に Docker がインストールされている必要があります。Amazon Braket Notebook には、Docker がプリインストールされているため、ホストされたノートブックでローカルジョブを即座にテストすることができます。ローカル環境に Docker をインストールするには、以下の[手順](https://docs.docker.com/get-docker/)を参照ください。ローカルジョブを初めて作成する場合、コンテナを構築する必要があるため、時間がかかります。その後の実行はより早くなります。ローカルジョブは Amazon Braket Console には表示されないことに注意してください。

ローカルでジョブを実行するには、Docker デーモンが起動していることを確認し、`AwsQuantumJob` の代わりに `LocalQuantumJob` を作成するだけです。ローカルジョブは常に同期的に実行され、ログが表示されます。

In [12]:
from braket.jobs.local.local_job import LocalQuantumJob

# This cell should take about 2 min
job = LocalQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Pulling docker container image. This may take a while.


Login Succeeded
1.0-cpu-py39-ubuntu22.04: Pulling from amazon-braket-base-jobs
74ac377868f8: Pulling fs layer
7fe9b8f01457: Pulling fs layer
780db72835a0: Pulling fs layer
c48699d0db25: Pulling fs layer
eacfd7fc3878: Pulling fs layer
d032eab7e852: Pulling fs layer
354743fa6126: Pulling fs layer
740069ceea3b: Pulling fs layer
d31f1b3318e6: Pulling fs layer
d6075f43bf82: Pulling fs layer
eb0da417065f: Pulling fs layer
6c747b12e5cf: Pulling fs layer
14facc2388ef: Pulling fs layer
ea3a15f84b42: Pulling fs layer
fd38d897c417: Pulling fs layer
3d6e4b48f11e: Pulling fs layer
c48699d0db25: Waiting
eacfd7fc3878: Waiting
d032eab7e852: Waiting
354743fa6126: Waiting
740069ceea3b: Waiting
d31f1b3318e6: Waiting
d6075f43bf82: Waiting
eb0da417065f: Waiting
6c747b12e5cf: Waiting
14facc2388ef: Waiting
ea3a15f84b42: Waiting
3d6e4b48f11e: Waiting
fd38d897c417: Waiting
780db72835a0: Verifying Checksum
780db72835a0: Download complete
74ac377868f8: Verifying Checksum
74ac377868f8: Download complete
c48699d0d

Using the short-lived AWS credentials found in session. They might expire while running.


Boto3 Version:  1.26.64
Beginning Setup
Checking for Additional Requirements
Additional Requirements Check Finished
Running Code As Subprocess
Test job started!!!!!
Counter({'0': 86, '1': 14})
Metrics - timestamp=1684037687.144999; braket_tasks_cost=0.00375; iteration_number=0;
Counter({'0': 96, '1': 4})
Metrics - timestamp=1684037689.732998; braket_tasks_cost=0.0075; iteration_number=1;
Counter({'0': 92, '1': 8})
Metrics - timestamp=1684037692.3552556; braket_tasks_cost=0.01125; iteration_number=2;
Counter({'1': 52, '0': 48})
Metrics - timestamp=1684037695.0975327; braket_tasks_cost=0.015; iteration_number=3;
Counter({'1': 90, '0': 10})
Metrics - timestamp=1684037697.72457; braket_tasks_cost=0.01875; iteration_number=4;
Test job completed!!!!!
Code Run Finished
c00555bbebdf8f543ff74f8e7bc998c5c433d7a08ef29d6d4371bdd3d728cee0


In [13]:
print("Task Summary")
print(job.result()['task summary'])
print('Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).')
print(f"Estimated cost to run tasks in this job: {job.result()['estimated cost']} USD")

Task Summary
{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 500, 'tasks': {'COMPLETED': 5}, 'execution_duration': 0.125, 'billed_execution_duration': 15.0}}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run tasks in this job: 0.01875 USD


## Braket コンソール上でジョブを作成する

`AwsQuantumJob.create` を使用してプログラムで Braket Job を作成する以外に、Braket コンソールでジョブを作成するオプションがあります。[こちらのリンク](https://us-west-2.console.aws.amazon.com/braket/home#/job/create) を参照して、\"Create job\" ページをご確認ください。まず、新しいジョブにユニークな名前をつけます。デフォルトでは、Amazon Braket は AmazonBraketJobsExecutionRole に、ジョブ実行に必要なすべての権限が存在することを確認します。ジョブのすべての入力と出力が保存されるデフォルトの S3 フォルダは、`amazon-braket-<region>-<account number>/jobs/<job-name>` という形式です。S3 バケットとフォルダが存在しない場合、Amazon Braket がお客様の代わりに作成します。これらのデフォルト設定は、\"Advanced settings\" タブでカスタマイズすることができます。

<div align="center"><img src="console_figures/1-create.png"/></div>

次に、ジョブのアルゴリズムスクリプトを選択します。スクリプトは、単一のファイルとしてコンソールで直接アップロードすることができます。アルゴリズムスクリプトに付随するヘルパー関数やその他の依存関係など、多くのファイルがある場合は、S3 バケットにファイルをアップロードして S3 フォルダを提供するオプションがあります。

<div align="center"><img src="console_figures/2-algorithm.png"/></div>

次に、ジョブに使用する Braket シミュレータまたは QPU の設定を行います。最後に、ハイパーパラメータとデータの入力、チェックポイントと出力データのデフォルトの場所をカスタマイズできます。これらの使用例については、他のノートブックの例で詳しく説明します。全ての設定を終えたら、\"Create job\" ボタンをクリックして、ジョブを作成します。これで、Braket コンソールでジョブ状態を確認できるようになりました。

<div align="center"><img src="console_figures/4-execution.png"/></div>

## まとめ

このチュートリアルでは、Amazon Braket SDK を使用し、5 つのシンプルな回路で最初の Braket Job を作成しました。また、ジョブの Amazon S3 フォルダと AWS リージョンを変更する方法と、結果を保存する方法を学びました。シミュレータや QPU で実行するためにデバイスをシームレスに変更する方法を学びました。ローカルモードを使って、コードを素早くテストしました。最後に、Braket コンソールを使って同じジョブを作成しました。